# World Development Indicators
This notebooks performs the required data preprocessing steps for the exploratory data analysis for the three world development indicators, namely: Access to electricity, Hospital beds and Gross Domestic Product. The data was retrieved from: [DataBankWorld Development Indicators](https://databank.worldbank.org/source/world-development-indicators#).
![World](https://upload.wikimedia.org/wikipedia/commons/0/06/Colour_World_Banner.png)

In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import warnings

In [2]:
# Read the exported csv.
wdi_raw_df = pd.read_csv("data/raw_data.csv", low_memory=False)
wdi_raw_df.tail(6)

,Country Name,Country Code,Series Name,Series Code,2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021]
284,Vietnam,VNM,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,2.91,..,2.5,3.18,2.6,..,..,..,..,..,..,..
285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289,Last Updated: 09/16/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
wdi_raw_df.dropna(inplace=True)
warnings.filterwarnings('ignore')

In [4]:
# Location data from Google was retrieved just in case we may need it in the future for visualization.
# Source: https://developers.google.com/public-data/docs/canonical/countries_csv
loc_df = pd.read_csv("data/loc.csv", low_memory=False)
loc_df.tail(2)

,country,latitude,longitude,name
243,ZM,-13.133897,27.849332,Zambia
244,ZW,-19.015438,29.154857,Zimbabwe


In [5]:
# Country details were also retrieved online
# Source: https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes
countries_df = pd.read_csv("data/countries.csv", low_memory=False)
countries_df.tail(2)

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
247,Zambia,ZM,ZMB,894,ISO 3166-2:ZM,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0
248,Zimbabwe,ZW,ZWE,716,ISO 3166-2:ZW,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0


In [6]:
# We combine all the dataset that we have about the details of the countries

country_details_df = pd.DataFrame(wdi_raw_df['Country Name'].unique(), columns=['countries'])
country_details_df['code3'] = wdi_raw_df['Country Code'].unique()

country_details_df = pd.merge(country_details_df, countries_df[['alpha-2','alpha-3','region','sub-region','name']], left_on='code3', right_on='alpha-3', how='left')
country_details_df.drop(["name", "code3"],axis=1, inplace=True)

country_details_df = pd.merge(country_details_df, loc_df, left_on='alpha-2', right_on='country', how = 'left')
country_details_df.drop(["name"],axis=1, inplace=True)

country_details_df

,countries,alpha-2,alpha-3,region,sub-region,country,latitude,longitude
0,Albania,AL,ALB,Europe,Southern Europe,AL,41.153332,20.168331
1,American Samoa,AS,ASM,Oceania,Polynesia,AS,-14.270972,-170.132217
2,Andorra,AD,AND,Europe,Southern Europe,AD,42.546245,1.601554
3,Armenia,AM,ARM,Asia,Western Asia,AM,40.069099,45.038189
4,Australia,AU,AUS,Oceania,Australia and New Zealand,AU,-25.274398,133.775136
...,...,...,...,...,...,...,...,...
90,Ukraine,UA,UKR,Europe,Eastern Europe,UA,48.379433,31.165580
91,United Kingdom,GB,GBR,Europe,Northern Europe,GB,55.378051,-3.435973
92,Uzbekistan,UZ,UZB,Asia,Central Asia,UZ,41.377491,64.585262
93,Vanuatu,VU,VUT,Oceania,Melanesia,VU,-15.376706,166.959158


In [7]:
country_details_df.to_csv("data/country_details_df.csv",  index=False)

In [8]:
# Rename column names to proper year and remove spaces
for col in wdi_raw_df.columns:
    if col.startswith('20'):
        wdi_raw_df.rename(columns={col:col[0:4]},inplace=True)
wdi_raw_df.columns = wdi_raw_df.columns.str.replace(' ', '')
wdi_raw_df.head(5)

,CountryName,CountryCode,SeriesName,SeriesCode,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Albania,ALB,Access to electricity (% of population),EG.ELC.ACCS.ZS,100,100,99.9000015258789,100,99.9499969482422,99.9800033569336,99.8899993896484,99.8899993896484,100,100,100,..
1,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,11926922828.9911,12890764531.3284,12319830437.3467,12776220507.0162,13228147516.1168,11386850129.8411,11861199830.8396,13019689336.6919,15156432309.8977,15401830754.0773,15131866270.5936,18260043499.8068
2,Albania,ALB,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,2.99,2.88,2.88,2.89,..,..,..,..,..,..,..,..
3,American Samoa,ASM,Access to electricity (% of population),EG.ELC.ACCS.ZS,..,..,..,..,..,..,..,..,..,..,..,..
4,American Samoa,ASM,GDP (current US$),NY.GDP.MKTP.CD,573000000,570000000,640000000,638000000,643000000,673000000,671000000,612000000,639000000,648000000,709000000,..


In [9]:
# Check for unique SeriesCode
wdi_raw_df['SeriesCode'].unique()

array(['EG.ELC.ACCS.ZS', 'NY.GDP.MKTP.CD', 'SH.MED.BEDS.ZS'], dtype=object)

In [10]:
# Extract Gross Domestic Product
gdp_df = wdi_raw_df.copy(deep=True)
gdp_df = gdp_df.query("SeriesCode == 'NY.GDP.MKTP.CD'")
gdp_df.reset_index(inplace=True, drop=True)
gdp_df.drop(columns=["SeriesName", "SeriesCode"],axis=1, inplace=True)
gdp_df.to_csv("data/gdp_df.csv",  index=False)
gdp_df.drop(columns=["CountryName"],axis=1, inplace=True)

In [11]:
# Extract Hospital beds (per 1000 people)
hbd_df = wdi_raw_df.copy(deep=True)
hbd_df = hbd_df.query("SeriesCode == 'SH.MED.BEDS.ZS'")
hbd_df.reset_index(inplace=True, drop=True)
hbd_df.drop(columns=["SeriesName", "SeriesCode"],axis=1, inplace=True)
hbd_df.to_csv("data/hbd_df.csv",  index=False)
hbd_df.drop(columns=["CountryName"],axis=1, inplace=True)

In [12]:
# Extract Access to electricity
ate_df = wdi_raw_df.copy(deep=True)
ate_df = ate_df.query("SeriesCode == 'EG.ELC.ACCS.ZS'")
ate_df.reset_index(inplace=True, drop=True)
ate_df.drop(columns=["SeriesName", "SeriesCode"],axis=1, inplace=True)
ate_df.to_csv("data/ate_df.csv",  index=False)
ate_df.drop(columns=["CountryName"],axis=1, inplace=True)

In [13]:
# Add column prefixes so when merging, it wouldn't be too confusing
for col in gdp_df.columns:
    if col.startswith('20'):
        gdp_df.rename(columns={col:'gdp_'+col},inplace=True)
for col in hbd_df.columns:
    if col.startswith('20'):
        hbd_df.rename(columns={col:'hbd_'+col},inplace=True)
for col in ate_df.columns:
    if col.startswith('20'):
        ate_df.rename(columns={col:'ate_'+col},inplace=True)

In [14]:
gdp_df.tail(2)

,CountryCode,gdp_2010,gdp_2011,gdp_2012,gdp_2013,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021
93,VUT,670713208.084552,770153304.153832,747839697.746593,758304466.245339,772315721.265827,730870581.672312,780889605.899977,880062103.045053,914736985.430944,936526267.622513,896827873.114925,983469256.849629
94,VNM,147198853578.923,172595960457.831,195592471672.748,213710932339.105,233449971628.523,239257234710.757,257095955975.529,281353402175.236,308702086757.617,330391329475.734,343242570827.351,362637524070.969


In [15]:
hbd_df.tail(2)

,CountryCode,hbd_2010,hbd_2011,hbd_2012,hbd_2013,hbd_2014,hbd_2015,hbd_2016,hbd_2017,hbd_2018,hbd_2019,hbd_2020,hbd_2021
93,VUT,..,..,..,..,..,..,..,..,..,..,..,..
94,VNM,2.91,..,2.5,3.18,2.6,..,..,..,..,..,..,..


In [16]:
ate_df.tail(2)

,CountryCode,ate_2010,ate_2011,ate_2012,ate_2013,ate_2014,ate_2015,ate_2016,ate_2017,ate_2018,ate_2019,ate_2020,ate_2021
93,VUT,44.0999984741211,41.9353294372559,43.7895278930664,31.7000007629395,49.057445526123,52.2051010131836,57.8199996948242,62.7999992370605,61.7545127868652,64.5901870727539,67.3332672119141,..
94,VNM,97.4300003051758,99,97.8899993896484,98.5549621582031,99.1999969482422,99.8213043212891,99.1999969482422,100,100,99.4000015258789,100,..


In [17]:
wdi_df = pd.read_csv("data/country_details_df_filled.csv", low_memory=False)
wdi_df

,countries,alpha-2,alpha-3,region,sub-region,latitude,longitude
0,Albania,AL,ALB,Europe,Southern Europe,41.153332,20.168331
1,American Samoa,AS,ASM,Oceania,Polynesia,-14.270972,-170.132217
2,Andorra,AD,AND,Europe,Southern Europe,42.546245,1.601554
3,Armenia,AM,ARM,Asia,Western Asia,40.069099,45.038189
4,Australia,AU,AUS,Oceania,Australia and New Zealand,-25.274398,133.775136
...,...,...,...,...,...,...,...
90,Ukraine,UA,UKR,Europe,Eastern Europe,48.379433,31.165580
91,United Kingdom,GB,GBR,Europe,Northern Europe,55.378051,-3.435973
92,Uzbekistan,UZ,UZB,Asia,Central Asia,41.377491,64.585262
93,Vanuatu,VU,VUT,Oceania,Melanesia,-15.376706,166.959158


In [18]:
# Merge Gross Domestic Product data to Country details
wdi_df = wdi_df.merge(gdp_df, how='left', left_on='alpha-3', right_on='CountryCode')

In [19]:
# Merge Hospital beds data to Country details
wdi_df = wdi_df.merge(hbd_df, how='left', left_on='alpha-3', right_on='CountryCode')

In [20]:
# Merge Access to electricity data
wdi_df = wdi_df.merge(ate_df, how='left', left_on='alpha-3', right_on='CountryCode')

In [21]:
wdi_df.drop(columns=["CountryCode","CountryCode_y", "CountryCode_x"],axis=1, inplace=True)

In [22]:
wdi_df = wdi_df.replace('..', 0)
wdi_df.tail(5).T

,90,91,92,93,94
countries,Ukraine,United Kingdom,Uzbekistan,Vanuatu,Vietnam
alpha-2,UA,GB,UZ,VU,VN
alpha-3,UKR,GBR,UZB,VUT,VNM
region,Europe,Europe,Asia,Oceania,Asia
sub-region,Eastern Europe,Northern Europe,Central Asia,Melanesia,South-eastern Asia
latitude,48.379433,55.378051,41.377491,-15.376706,14.058324
longitude,31.16558,-3.435973,64.585262,166.959158,108.277199
gdp_2010,141209864408.488,2491110093454.18,49765676402.4495,670713208.084552,147198853578.923
gdp_2011,169333048847.834,2674891473843.81,60178909297.208,770153304.153832,172595960457.831
gdp_2012,182592416468.527,2719158341005.79,67517349212.0609,747839697.746593,195592471672.748


In [23]:
wdi_df.dtypes

countries      object
alpha-2        object
alpha-3        object
region         object
sub-region     object
latitude      float64
longitude     float64
gdp_2010       object
gdp_2011       object
gdp_2012       object
gdp_2013       object
gdp_2014       object
gdp_2015       object
gdp_2016       object
gdp_2017       object
gdp_2018       object
gdp_2019       object
gdp_2020       object
gdp_2021       object
hbd_2010       object
hbd_2011       object
hbd_2012       object
hbd_2013       object
hbd_2014       object
hbd_2015       object
hbd_2016       object
hbd_2017       object
hbd_2018       object
hbd_2019       object
hbd_2020        int64
hbd_2021        int64
ate_2010       object
ate_2011       object
ate_2012       object
ate_2013       object
ate_2014       object
ate_2015       object
ate_2016       object
ate_2017       object
ate_2018       object
ate_2019       object
ate_2020       object
ate_2021        int64
dtype: object

In [24]:
# Change data type to Float
for col in wdi_df.columns:
    if col[4:6] == '20':
        wdi_df[col] = wdi_df[col].astype(dtype = 'float64')

In [25]:
wdi_df.dtypes

countries      object
alpha-2        object
alpha-3        object
region         object
sub-region     object
latitude      float64
longitude     float64
gdp_2010      float64
gdp_2011      float64
gdp_2012      float64
gdp_2013      float64
gdp_2014      float64
gdp_2015      float64
gdp_2016      float64
gdp_2017      float64
gdp_2018      float64
gdp_2019      float64
gdp_2020      float64
gdp_2021      float64
hbd_2010      float64
hbd_2011      float64
hbd_2012      float64
hbd_2013      float64
hbd_2014      float64
hbd_2015      float64
hbd_2016      float64
hbd_2017      float64
hbd_2018      float64
hbd_2019      float64
hbd_2020      float64
hbd_2021      float64
ate_2010      float64
ate_2011      float64
ate_2012      float64
ate_2013      float64
ate_2014      float64
ate_2015      float64
ate_2016      float64
ate_2017      float64
ate_2018      float64
ate_2019      float64
ate_2020      float64
ate_2021      float64
dtype: object

In [26]:
# Export all data
wdi_df.to_csv("data/wdi_df.csv",  index=False)

In [27]:
gdp_df = pd.read_csv("data/gdp_df.csv", low_memory=False)
gdp_df = gdp_df.replace('..', 0)
for col in gdp_df.columns:
    if col.startswith('20'):
        gdp_df[col] = gdp_df[col].astype(dtype = 'float64')
gdp_df.to_csv("data/gdp_df.csv",  index=False)

In [28]:
hbd_df = pd.read_csv("data/hbd_df.csv", low_memory=False)
hbd_df = hbd_df.replace('..', 0)
for col in hbd_df.columns:
    if col.startswith('20'):
        hbd_df[col] = hbd_df[col].astype(dtype = 'float64')
hbd_df.to_csv("data/hbd_df.csv",  index=False)

In [29]:
ate_df = pd.read_csv("data/ate_df.csv", low_memory=False)
ate_df = ate_df.replace('..', 0)
for col in ate_df.columns:
    if col.startswith('20'):
        ate_df[col] = ate_df[col].astype(dtype = 'float64')
ate_df.to_csv("data/ate_df.csv",  index=False)

In [30]:
gdp_df.head(5)

,CountryName,CountryCode,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Albania,ALB,1.192692e+10,1.289076e+10,1.231983e+10,1.277622e+10,1.322815e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.515643e+10,1.540183e+10,1.513187e+10,1.826004e+10
1,American Samoa,ASM,5.730000e+08,5.700000e+08,6.400000e+08,6.380000e+08,6.430000e+08,6.730000e+08,6.710000e+08,6.120000e+08,6.390000e+08,6.480000e+08,7.090000e+08,0.000000e+00
2,Andorra,AND,3.449926e+09,3.629134e+09,3.188653e+09,3.193513e+09,3.271686e+09,2.789881e+09,2.896610e+09,3.000162e+09,3.218420e+09,3.155149e+09,2.891001e+09,3.329911e+09
3,Armenia,ARM,9.260285e+09,1.014211e+10,1.061932e+10,1.112147e+10,1.160951e+10,1.055334e+10,1.054614e+10,1.152746e+10,1.245794e+10,1.361929e+10,1.264121e+10,1.386118e+10
4,Australia,AUS,1.147589e+12,1.397908e+12,1.546509e+12,1.576335e+12,1.467505e+12,1.350534e+12,1.206685e+12,1.326883e+12,1.428530e+12,1.391953e+12,1.327836e+12,1.542660e+12


In [31]:
# Fill missing data with mean for now to be used in our PBI report
gdp_df = gdp_df.mask(gdp_df ==0).fillna(gdp_df.mean())
hbd_df = hbd_df.mask(hbd_df ==0).fillna(hbd_df.mean())
ate_df = ate_df.mask(ate_df ==0).fillna(ate_df.mean())

In [32]:
# Create a new data frame with Year as 1 column
gdp_one_df = pd.DataFrame(columns = ["CountryName", "CountryCode", "Year","GDP"])
gdp_one_df
for col in gdp_df.columns:
    if col.startswith('20'):
        gdp_new_df = pd.DataFrame()
        gdp_new_df["CountryName"] = gdp_df["CountryName"]
        gdp_new_df["CountryCode"] = gdp_df["CountryCode"]
        gdp_new_df["Year"] = col
        gdp_new_df["GDP"] = gdp_df[col]
        
        gdp_one_df = gdp_one_df.append(gdp_new_df)
         
gdp_one_df.to_csv("data/gdp_one_df.csv",  index=False)

In [33]:
# Create a new data frame with Year as 1 column
hbd_one_df = pd.DataFrame(columns = ["CountryName", "CountryCode", "Year","HBD"])
hbd_one_df

for col in hbd_df.columns:
    if col.startswith('20'):
        hbd_new_df = pd.DataFrame()
        hbd_new_df["CountryName"] = hbd_df["CountryName"]
        hbd_new_df["CountryCode"] = hbd_df["CountryCode"]
        hbd_new_df["Year"] = col
        hbd_new_df["HBD"] = hbd_df[col]
        
        hbd_one_df = hbd_one_df.append(hbd_new_df)
         
hbd_one_df.to_csv("data/hbd_one_df.csv",  index=False)

In [34]:
# Create a new data frame with Year as 1 column
ate_one_df = pd.DataFrame(columns = ["CountryName", "CountryCode", "Year","ATE"])
ate_one_df

for col in ate_df.columns:
    if col.startswith('20'):
        ate_new_df = pd.DataFrame()
        ate_new_df["CountryName"] = ate_df["CountryName"]
        ate_new_df["CountryCode"] = ate_df["CountryCode"]
        ate_new_df["Year"] = col
        ate_new_df["ATE"] = ate_df[col]
        
        ate_one_df = ate_one_df.append(ate_new_df)
         
ate_one_df.to_csv("data/ate_one_df.csv",  index=False)